In [1]:
%matplotlib notebook
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

In [2]:
x1 = np.float32(np.random.uniform(low = -10, high = 10, size = 100))
x2 = np.float32(np.random.uniform(low = -10, high = 10, size = 100))
err = np.float32(np.random.normal(loc = 0, scale = 5, size = 100))

In [3]:
y = -40 + 10 * x1 + 20 * x2 + err

In [4]:
data = pd.DataFrame({
    ('features', 'x1'): x1,
    ('features', 'x2'): x2,
    ('labels', 'y'): y
})

In [5]:
tf.reset_default_graph()

In [6]:
features, labels = (
   tf.data.Dataset.from_tensor_slices((data['features'].values, data['labels'].values))
                  .shuffle(buffer_size = 50)
                  .repeat()
                  .batch(batch_size = 32)
                  .make_one_shot_iterator()
                  .get_next()
)

In [7]:
input_shape = features.shape
weight = tf.get_variable('weight', shape = [input_shape[1], 1], initializer = tf.random_normal_initializer())
bias = tf.get_variable('bias', shape = [1], initializer = tf.random_normal_initializer())
predictions = tf.nn.xw_plus_b(features, weight, bias)

In [8]:
loss = tf.losses.mean_squared_error(labels = labels, predictions = predictions)

In [9]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.1)
train_op = optimizer.minimize(loss, global_step = tf.train.get_or_create_global_step())

In [10]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for train_idx in range(1000):
        sess.run(train_op)
    print(sess.run([weight, bias]))

[array([[10.100036],
       [20.09031 ]], dtype=float32), array([-39.95981], dtype=float32)]
